In [1]:
# !pip install py2neo

In [2]:
import py2neo  

In [8]:
path = "bolt://localhost:7687"
auth_info = ("neo4j","baseball5757")

In [9]:
from py2neo import Graph 

In [14]:
our_graph = Graph(path, auth=auth_info)

In [15]:
our_graph

Graph('bolt://neo4j@localhost:7687')

In [13]:
# Add a node to the graph 
our_graph.run("CREATE(:SCHOOL{name:'VT',type:'public'})")

(No data)

In [16]:
# Add another node to the graph 
our_graph.run("CREATE(:SCHOOL{name:'UVA',type:'public'})")

(No data)

In [24]:
# Delete EVERYTHING in the graph
our_graph.run("MATCH (n) DETACH DELETE n")

(No data)

In [32]:
# Delete EVERYTHING in the graph
our_graph.run("MATCH (n) DETACH DELETE n")

# Add nodes to the graph 
our_graph.run("CREATE(:SCHOOL{name:'VT',type:'public'})")
our_graph.run("CREATE(:SCHOOL{name:'UVA',type:'public'})")
our_graph.run("CREATE(:COMPANY{name:'FORGE',type:'501c'})")

# Create a relationship between nodes 
our_graph.run("MATCH (n:SCHOOL{name:'UVA'}), (c:COMPANY{name:'FORGE'}) CREATE (n)-[:partners_with]->(c)")
our_graph.run("MATCH (n:SCHOOL{name:'UVA'}), (m:SCHOOL{name:'VT'}) CREATE (n)-[:rivals_with]->(m)")
our_graph.run("MATCH (n:SCHOOL{name:'UVA'}), (m:SCHOOL{name:'VT'}) CREATE (m)-[:rivals_with]->(n)")

# Add information to the relationships 
our_graph.run("MATCH (n:SCHOOL{name:'UVA'}) SET n.founded_in = 1819")

# Add founded_in field to the UVA node and the VT node 
our_graph.run("MATCH (n:SCHOOL{name:'UVA'}) SET n.founded_in = 1819")
our_graph.run("MATCH (n:SCHOOL{name:'VT'}) SET n.founded_in = 1872")

# Remove a property from the FORGE node 
our_graph.run("MATCH (n:COMPANY{name:'FORGE'}) REMOVE n.type")

(No data)

In [33]:
from py2neo import Node, Relationship 
from py2neo.matching import *
nodes = NodeMatcher(our_graph)

In [45]:
# Obtain node references 
forge = nodes.match(name="FORGE").first()
uva = nodes.match(name="UVA").first()

# Create a Kaleigh node 
kaleigh = Node("Person", name='Kaleigh Watson', coworkers=['Andy','Amanda','Daniel'])
our_graph.create(kaleigh)

# Define relationships
WorksFor = Relationship.type('WORKS_FOR')
Attended = Relationship.type('ATTENDED')

# Set relationships between kaleigh and forge and uva
our_graph.create(WorksFor(kaleigh, forge))
our_graph.create(Attended(kaleigh, uva))

Our Project

In [118]:
# Import libraries of interest 
import py2neo  
from py2neo import Graph 
import pandas as pd 
import numpy as np 
from py2neo import Node, Relationship 
from py2neo.matching import *

In [119]:
# Set up path and auth info 
path = "bolt://localhost:7687"
auth_info = ("neo4j","launch")

courses_path = "C:/Users/Zachery Key/Desktop/Forge/COURSES.csv"
school_path = "C:/Users/Zachery Key/Desktop/Forge/SCHOOLS.csv"
prof_path = "C:/Users/Zachery Key/Desktop/Forge/PROFS.csv"
city_path = "C:/Users/Zachery Key/Desktop/Forge/CITIES.csv"
student_path = "C:/Users/Zachery Key/Desktop/Forge/STUDENTS.csv"

In [120]:
# Initialize graph object in neo4j
our_graph = Graph(path, auth=auth_info)
# Initialze node matcher object 
nodes = NodeMatcher(our_graph)

In [129]:
# Import data 
COURSE = pd.read_csv(courses_path)
SCHOOL = pd.read_csv(school_path)
PROF = pd.read_csv(prof_path, encoding= 'unicode_escape')
CITY = pd.read_csv(city_path)
STUDENT = pd.read_csv(student_path)

In [148]:
# Delete EVERYTHING in the graph
our_graph.run("MATCH (n) DETACH DELETE n")

(No data)

In [149]:
# Professor 
for i in range(len(PROF)):
    row = PROF.iloc[i]
    person = Node("Professor",
                  prof_id = int(row['Prof_ID\xa0']),
                  name = row['Professor Name'], 
                  tenure = bool(row['Tenure']),
                  school_id = row['School_ID'],
                  city_id = int(row['City_ID'])
                 )
    our_graph.create(person)

# Schools 
for i in range(len(SCHOOL)):
    row = SCHOOL.iloc[i]
    school = Node("School",
                  name = row['name'],
                  size = int(row['size ']),
                  year_founded = int(row['year_founded']),
                  num_majors = int(row['num_majors']),
                  dean_chair = row['dean_chair'],
                  school_id = row['school_ID']
                 )
    our_graph.create(school)

# Courses
for i in range(len(COURSE)):
    row = COURSE.iloc[i]
    courses = Node(
                "Course", 
                name = row['COURSE ID '],
                credits = int(row['CREDITS']),
                school_id = row['SCHOOL_ID'],
                prof_id = int(row['PROF_ID'])
                 )
    our_graph.create(courses)

# Students
for i in range(len(STUDENT)):
    row = STUDENT.iloc[i]
    student = Node(
                 "Student", 
                 name = row['first_name'] + ' ' + row['last_name'],
                 first_name = row['first_name'],
                 last_name = row['last_name'],
                 birthdate = row['dob'],
                 computing_id = row['computing_id'], 
                 school_id = row['school_id'],
                 city_id = int(row['city_id']),
                 fav_course = row['fav_course']
                 )
    our_graph.create(student)

CITY['pop_size'] = CITY.pop_size.str.replace(',','')

# City
for i in range(len(CITY)):
    row = CITY.iloc[i]
    city = Node("City",
                city_id = int(row['city_id']),
                name = row['city'],
                state = row['state'],
                football_team = row['football'],
                basketball_team = row['basketball'],
                hockey_team = row['hockey'],
                baseball_team = row['baseball'],
                population = int(row['pop_size']),
                zipcode = int(row['zipcode'])
                 )
    our_graph.create(city)

In [150]:
# Create relationship between all students and schools 
our_graph.run("MATCH (n:Student), (m:School) WHERE n.school_id = m.school_id CREATE (n)-[:ATTENDED]->(m)")

# Create relationship between students and favorite courses 
our_graph.run("MATCH (n:Student), (m:Course) WHERE n.fav_course = m.name CREATE (n)-[:FAVORITE_COURSE_IS]->(m)")

# Create relationship between professors and courses 
our_graph.run("MATCH (n:Professor), (m:Course) WHERE n.prof_id = m.prof_id CREATE (n)-[:TEACHES]->(m)")

(No data)

(No data)